<a href="https://colab.research.google.com/github/razsaroya/Generate-abstract-art-StyleGAN/blob/main/SG2_ADA_PyTorch_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating abstract art with StyleGAN2-ADA (PyTorch)

## Setup

Connect to Google Drive account

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install stylegan repository

The next cell will install the StyleGAN repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O 
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex

In [ ]:
%cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets"
!mkdir pretrained
%cd pretrained
!gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU

## Dataset Preparation

Upload a .zip of square images to the `datasets` folder

## Train model

Training parameters

* `dataset_path`: this is the path to your .zip file
* `resume_from`: if you’re starting a new dataset I recommend `'./pretrained/wikiart.pkl'`
* `mirror_x` and `mirror_y`: Allow the dataset to use horizontal or vertical mirroring.

In [ ]:
#required: 
%cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
dataset_path = './datasets/datasetV2.zip'
resume_from = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl'
aug_strength = 0.0
train_count = 0
mirror_x = True

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
snapshot_count = 2

In [ ]:
!pip3 install ninja

In [ ]:
!python train.py \
--gpus=1 \
--cfg=$config \
--metrics=None \
--outdir=./results \
--resume=$resume_from \
--data=$dataset_path \
--snap=$snapshot_count \
--augpipe=$augs \
--initstrength=$aug_strength \
--gamma=$gamma_value \
--mirror=$mirror_x \
--mirrory=False \
--nkimg=$train_count

### Resume Training

Once Colab has shutdown, you’ll need to resume your training. Reset the variables above, particularly the `resume_from` and `aug_strength` settings.

1. Point `resume_from` to the last .pkl you trained (you’ll find these in the `results` folder)
2. Update `aug_strength` to match the augment value of the last pkl file. Often you’ll see this in the console, but you may need to look at the `log.txt`. Updating this makes sure training stays as stable as possible.
3. You may want to update `train_count` to keep track of your training progress.

Once all of this has been reset, run that variable cell and the training command cell after it.

## Testing/Inference

Also known as "Inference", "Evaluation" or "Testing" the model. This is the process of usinng your trained model to generate new material, usually images or videos.

### Generate Single Images

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`: This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation`: Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 


In [ ]:
!python generate.py \
--outdir="/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00001-datasetV2-mirror-11gb-gpu-gamma50-bg-resumecustom/out/616" \
--trunc=0.8 \
--seeds=0-10000 \
--network="/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00001-datasetV2-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000616.pkl"

### Truncation Traversal

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

#### Options 
`--network`: Again, this should be the path to your .pkl file.

`--seeds`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)


In [ ]:
!python generate.py --process="truncation" --outdir=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00005-images-mirror-11gb-gpu-gamma50-bg-resumecustom/out/trunc-trav-3/ --start=-0.8 --stop=2.8 --increment=0.01 --seeds=124 --network=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00005-images-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000032.pkl

### Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at different examples of interpolation below.

#### Options

`--network`: path to your .pkl file

`--interpolation`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value

# Projecting

In [ ]:
!pip3 install ninja

In [ ]:
%cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/"
!python projector.py \
--network="/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00001-datasetV2-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000616.pkl" \
--target="/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00001-datasetV2-mirror-11gb-gpu-gamma50-bg-resumecustom/out/616_projecting/Coca-Cola_logo.png" \
--num-steps=100 \
--seed=100 \
--save-video=true \
--outdir="/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00001-datasetV2-mirror-11gb-gpu-gamma50-bg-resumecustom/out/616_projecting"

#### Linear Interpolation

In [ ]:
!python generate.py --outdir=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00005-images-mirror-11gb-gpu-gamma50-bg-resumecustom/out/video1-w-0.5/ --space="z" --trunc=0.5 --process="interpolation" --seeds=126,169 --network=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00005-images-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000032.pkl --frames=48

In [ ]:
!python generate.py --outdir=out/video1-w/ --space="w" --trunc=1 --process="interpolation" --seeds=3,10,27,33 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

In [ ]:
!zip -r vid1.zip /content/out/video1-w-0.5

#### Slerp Interpolation

This gets a little heady, but technically linear interpolations are not the best in high-dimensional GANs. [This github link](https://github.com/soumith/dcgan.torch/issues/14) is one of the more popular explanations ad discussions.

In reality I do not find a huge difference between linear and spherical interpolations (the difference in z- and w-space is enough in many cases), but I’ve implemented slerp here for anyone interested.

Note: Slerp in w space currently isn’t supported. I’m working on it.

In [ ]:
!python generate.py --outdir=out/video1/ --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

#### Noise Loop

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

`--interpolation="noiseloop"`: set this to use the noise loop funtion

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

Noise loops currently only work in z space.

In [ ]:
!python generate.py --outdir=out/video-noiseloop-0.9d/ --trunc=0.8 --process="interpolation" --interpolation="noiseloop" --diameter=0.9 --random_seed=100 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

#### Circular Loop

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

In [ ]:
!python generate.py --outdir=out/video-circularloop/ --trunc=1 --process="interpolation" --interpolation="circularloop" --diameter=800.00 --frames=720 --random_seed=90 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

## Combine NPZ files together

In [ ]:
!python combine_npz.py --outdir=/content/npz --npzs='/content/pb-proj1-3-clip.npz,/content/pb-proj1-clip-nocenter.npz,/content/pb-proj2-clip-nocenter.npz,/content/projected_w (2).npz'